**Разбиение выборки** - это разделение имеющихся данных на несколько частей для проведения процессов обучения и оценки качества алгоритма машинного обучения на полностью независимых наборах данных.

### Способы разбиения выборки
1. **hold-out** разбиение (отложенная выборка, т.е. на две части - обучающую и тестовую)
2. **cross-validation** разбиение (на k наборов, имеющих обучающую и тестовую части)

    #### Виды разбиений для перекрёстной прорверки:
    1. **k-fold** (разбиение на k частей с примерно одинаковым количеством объектов в каждом, из которых получают k наборов обучающих и
    тестовых выборок, в которых одна часть выступает в роли тестовой выборки, а объединение остальных — в роли обучающей)
    2. **Stratified k-fold**(k-fold разбиение со стратификацией, т.е. все наборы содержат примерно такой же процент объектов каждого
    класса, что и исходные данные)
    2. **leave-one-out** (отложенный пример)
    и др.

**Кросс-валидация** (перекрёстная проверка) — это процедура для оценки обобщающей способности алгоритмов, состоящая из следующих этапов:

1. Фиксируется число блоков k. 

2. Выборка разбивается на k блоков определённым образом и получается k наборов данных, в которых один блок выступает в роли тестовой выборки, а объединение остальных блоков — в роли тренировочной. 

3. Осуществляется k итераций обучения модели на тренировочной выборке и подсчёт метрик на тестовой.

4. Итоговые метрики получаются усреднением k получившихся результатов.

Импортируем необходимые библиотеки:

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, r2_score, mean_squared_error
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.datasets import load_iris, load_wine, load_diabetes

from sklearn.model_selection import KFold, StratifiedKFold, LeaveOneOut, cross_val_score, cross_validate

Загрузим встроенные данные об ирисах и выведем их описание:

## Семинар 12

# Тема: Кросс-валидация

In [2]:
iris = load_iris()
print(iris.DESCR)

.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

    :Number of Instances: 150 (50 in each of three classes)
    :Number of Attributes: 4 numeric, predictive attributes and the class
    :Attribute Information:
        - sepal length in cm
        - sepal width in cm
        - petal length in cm
        - petal width in cm
        - class:
                - Iris-Setosa
                - Iris-Versicolour
                - Iris-Virginica
                
    :Summary Statistics:

    ============== ==== ==== ======= ===== ====================
                    Min  Max   Mean    SD   Class Correlation
    ============== ==== ==== ======= ===== ====================
    sepal length:   4.3  7.9   5.84   0.83    0.7826
    sepal width:    2.0  4.4   3.05   0.43   -0.4194
    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
    petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)
    ============== ==== ==== ======= ===== ===========

Выделим признаки и целевую переменную:

In [3]:
X = iris.data
y = iris.target

Сделаем hold-out разбиение данных на обучающую и тестовую части:

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

Обучим модель логистической регрессии на обучающих данных и найдём значение метрики accuracy на тестовых.

In [5]:
model = LogisticRegression()
model.fit(X_train, y_train)
print('accuracy =', model.score(X_test, y_test))

accuracy = 1.0


/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Поскольку accuracy = 1, предполагаем, что модель всегда верно классифицирует данные. Проверим так ли это, проведя перекрёстную проверку.

Сделаем разбиение исходных данных на k-блоков с перемешиванием, где k = 3. Выведем индексы обучающих и тестовых частей каждого блока, а также их длины.

In [6]:
kf = KFold(n_splits = 3,shuffle=True, random_state=42)

for i, (train_index, test_index) in enumerate(kf.split(X)):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    print("Fold: {}, Длинна train: {}, Длинна test: {}".format(i, len(train_index), len(test_index)))
    print(f"  Train: index={train_index}")
    print(f"  Test:  index={test_index}")

Fold: 0, Длинна train: 100, Длинна test: 50
  Train: index=[  0   1   2   3   5   6   7   8  13  14  17  20  21  23  24  25  28  33
  34  35  37  38  39  40  41  43  44  46  47  48  49  50  52  53  54  57
  58  59  60  61  62  63  66  67  70  71  72  74  77  79  80  83  84  87
  88  89  90  91  92  93  94  95  96  97  98  99 100 101 102 103 105 106
 107 111 112 113 114 115 116 117 119 120 121 122 123 124 125 126 129 130
 134 135 136 138 139 140 144 147 148 149]
  Test:  index=[  4   9  10  11  12  15  16  18  19  22  26  27  29  30  31  32  36  42
  45  51  55  56  64  65  68  69  73  75  76  78  81  82  85  86 104 108
 109 110 118 127 128 131 132 133 137 141 142 143 145 146]
Fold: 1, Длинна train: 100, Длинна test: 50
  Train: index=[  1   2   3   4   6   9  10  11  12  14  15  16  17  18  19  20  21  22
  26  27  29  30  31  32  36  37  38  41  42  45  46  48  50  51  52  54
  55  56  57  58  59  61  63  64  65  68  69  71  72  73  74  75  76  78
  79  81  82  85  86  87  88  90  91 

Сделаем k-блочную перекрёстную проверку (k = 3) модели логистической регрессии для метрики accuracy, используя библиотечную функцию cross_val_score.

In [7]:
cv_results = cross_val_score(model,                  # модель
                             X,                      # матрица признаков
                             y,                      # вектор цели
                             cv = kf,                # тип разбиения (можно указать просто число фолдов cv = 3)
                             scoring = 'accuracy',   # метрика 'f1_macro','precision_macro', 'recall_macro'
                                                             # 'f1_micro','precision_micro', 'recall_micro'
                                                             # 'f1_weighted','precision_weighted', 'recall_weighted'
                             n_jobs=-1)              # используются все ядра CPU
cv_results

/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_ite

array([1.  , 0.92, 0.98])

Найдём среднее по кросс-валидации:

In [8]:
cv_results.mean()

0.9666666666666667

Делаем вывод, что модель классифицирует верно только 97% данных.

# Кросс-валидация в задаче классификации

### 1. Загрузите встроенный датасет o классификации вина из библиотеки sklearn.datasets. Выведите его описание. Обозначьте за _X_ двумерный numpy-массив признаков, а за _X_frame_ датафрейм признаков с соответствующими названиями колонок. Выведите размеры и первые пять строк, полученного датафрейма. Обозначьте за _y_ одномерный numpy-массив, а за _y_series_  - серию со значениями целевой переменной. Определите количество классов и количество объектов в каждом классе для целевой переменной. Убедитесь в том, что в данных нет пропущенных значений и они все числовые.
Указание: Для загрузки датасета [wine](https://scikit-learn.org/1.5/modules/generated/sklearn.datasets.load_wine.html) напишите wine = load_wine(). Для определения количества объектов в каждом классе используйте метод _.value_counts()_ библиотеки _pandas_.

In [9]:
wine = load_wine()

In [10]:
#print(wine.DESCR)

In [11]:
X = wine.data
y = wine.target
(X[:5,].shape, y[:5,].shape)

((5, 13), (5,))

In [12]:
X_frame = pd.DataFrame(X, columns=wine.feature_names)
X_frame

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740.0
174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0
175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840.0


In [13]:
y_series = pd.Series(y)

In [14]:
X_frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178 entries, 0 to 177
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   alcohol                       178 non-null    float64
 1   malic_acid                    178 non-null    float64
 2   ash                           178 non-null    float64
 3   alcalinity_of_ash             178 non-null    float64
 4   magnesium                     178 non-null    float64
 5   total_phenols                 178 non-null    float64
 6   flavanoids                    178 non-null    float64
 7   nonflavanoid_phenols          178 non-null    float64
 8   proanthocyanins               178 non-null    float64
 9   color_intensity               178 non-null    float64
 10  hue                           178 non-null    float64
 11  od280/od315_of_diluted_wines  178 non-null    float64
 12  proline                       178 non-null    float64
dtypes: fl

In [15]:
y_series.value_counts()

1    71
0    59
2    48
Name: count, dtype: int64

In [16]:
y_series.unique()

array([0, 1, 2])

### 2. Сделайте hold-out (на две части) разбиение данных на обучающую и тестовую части в соотношении 4:1 тремя способами: без перемешивания и с перемешиванием двумя разными способами. Выведите количество объектов, попавшых в обучающую и в тестовую части. Выведите на экран индексы объектов обучающей и тестовой частей для каждого разбиения.
Указание: Используйте функцию train_test_split. Для разбиение без перемешивания, укажите параметр shuffle=False. Для разбиений с перемешиванием, зафиксируйте разные способы перемешивания, указав random_state=0 и  random_state=1. Для выведения индексов объектов, попавших в обучающую и тестовую выборки, передавайте в функцию датафрейм и серию (не numpy-массивы), а индексы выводите при помощи .index библиотеки pandas.

In [17]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(X, y, shuffle=False, test_size=0.2)

In [18]:
(X_train1.shape, X_test1.shape)

((142, 13), (36, 13))

In [19]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y, random_state=0, shuffle=True, test_size=0.2)

In [20]:
(X_train2.shape, X_test2.shape)

((142, 13), (36, 13))

In [21]:
X_train3, X_test3, y_train3, y_test3 = train_test_split(X, y, random_state=1, shuffle=True, test_size=0.2)

In [22]:
(X_train3.shape, X_test3.shape)

((142, 13), (36, 13))

### 3.Обучите модель логистической регрессии LogisticRegression(max_iter=5000) на трёх наборах обучающих данных, полученных в предыдущем задании. Сделайте предсказания на соответствующих трёх наборах тестовых данных и вычислите значения метрики accuracy для каждого из наборов. Сделайте вывод о том, влияет ли способ разбиения на значение метрики.

In [23]:
model = LogisticRegression(max_iter=5000)
model.fit(X_train1, y_train1)
print('accuracy =', model.score(X_test1, y_test1))

accuracy = 1.0


In [24]:
model = LogisticRegression(max_iter=5000)
model.fit(X_train2, y_train2)
print('accuracy =', model.score(X_test2, y_test2))

accuracy = 0.9722222222222222


In [25]:
model = LogisticRegression(max_iter=5000)
model.fit(X_train3, y_train3)
print('accuracy =', model.score(X_test3, y_test3))

accuracy = 0.9444444444444444


### 4. Сделайте разбиение исходных данных на k-блоков с перемешиванием, где k = 5. Выведите индексы обучающих и тестовых частей каждого блока.

Указание: Используйте класс [KFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html) с параметрами n_splits = 5, shuffle=True, random_state=42.

In [26]:
kf = KFold(n_splits = 5,shuffle=True, random_state=42)

for i, (train_index, test_index) in enumerate(kf.split(X)):
    X_train1, X_test1 = X[train_index], X[test_index]
    y_train1, y_test1 = y[train_index], y[test_index]
    print("Fold: {}, Длинна train: {}, Длинна test: {}".format(i, len(train_index), len(test_index)))
    print(f"  Train: index={train_index}")
    print(f"  Test:  index={test_index}")

Fold: 0, Длинна train: 142, Длинна test: 36
  Train: index=[  0   1   2   3   4   5   6   7   8  10  11  13  14  17  20  21  22  23
  25  26  27  28  32  33  34  35  36  37  38  39  40  43  44  46  47  48
  49  50  51  52  53  54  56  57  58  59  61  62  63  64  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  83  84  85  86  87  88  89  91
  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108 110
 112 115 116 120 121 122 123 124 125 126 127 129 130 131 132 133 134 135
 136 137 138 139 142 143 144 146 147 148 149 151 152 153 154 155 156 157
 158 159 160 161 162 163 165 166 167 168 170 172 173 175 176 177]
  Test:  index=[  9  12  15  16  18  19  24  29  30  31  41  42  45  55  60  65  66  67
  82  90 109 111 113 114 117 118 119 128 140 141 145 150 164 169 171 174]
Fold: 1, Длинна train: 142, Длинна test: 36
  Train: index=[  0   1   3   5   7   8   9  10  12  13  14  15  16  17  18  19  20  21
  23  24  25  28  29  30  31  33  34  35  37  39  40  41  42  43  44  

### 5. Напишите свой код, осуществляющий k-блочную перекрёстную проверку (кросс-валидацию) с количеством блоков k = 5 модели логистической регрессии для метрики accuracy. Усредните полученный результат.

Указание: Используйте индексы блоков, полученных в предыдущем задании.

In [27]:
cv_results = cross_val_score(model,                  # модель
                             X,                      # матрица признаков
                             y,                      # вектор цели
                             cv = kf,                # тип разбиения (можно указать просто число фолдов cv = 3)
                             scoring = 'accuracy',   # метрика 'f1_macro','precision_macro', 'recall_macro'
                                                             # 'f1_micro','precision_micro', 'recall_micro'
                                                             # 'f1_weighted','precision_weighted', 'recall_weighted'
                             n_jobs=-1)              # используются все ядра CPU
cv_results

array([1.        , 0.91666667, 0.94444444, 0.97142857, 0.97142857])

In [28]:
cv_results.mean()

0.9607936507936508

### 6. Сделайте k-блочную перекрёстную проверку с k = 5 модели логистической регрессии для метрики accuracy, используя библиотечную функцию cross_val_score. Убедитесь, что получится тот же результат, что и в предыдущем задании. Также выведите дисперсию по кросс-валидации.
Указание: Функция [cross_val_score](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html) позволяет осуществлять перекрёстную проверку для вычисления только одной метрики, которую нужно указать в параметре scoring.

In [29]:
print(cross_val_score(model, X, y, cv=kf,scoring=('accuracy')).mean())

0.9607936507936508


### 7. Сделайте k-блочную перекрёстную проверку с k = 5 модели логистической регрессии для двух метрик accuracy и F1-мера, используя библиотечную функцию cross_validate. Выведите полученные массивы со значениями метрик на обучающих и тестовых данных. Усредните их значения.
Указание: Функция [cross_validate](https://scikit-learn.org/1.5/modules/generated/sklearn.model_selection.cross_validate.html) позволяет осуществлять перекрёстную проверку нескольких метрик, которые нужно указать списком в параметре scoring.

In [30]:
metrics = cross_validate(model, X, y, cv=kf, scoring=('accuracy', 'f1_macro'))
metrics

{'fit_time': array([0.38559318, 0.29584193, 0.36185598, 0.30921793, 0.32828903]),
 'score_time': array([0.00140977, 0.00129414, 0.00127101, 0.00129318, 0.00129795]),
 'test_accuracy': array([1.        , 0.91666667, 0.94444444, 0.97142857, 0.97142857]),
 'test_f1_macro': array([1.        , 0.9194847 , 0.94530444, 0.97079365, 0.97096189])}

In [31]:
metrics['test_accuracy'].mean()

0.9607936507936508

In [32]:
metrics['test_f1_macro'].mean()

0.9613089355857725

### 8. Сделайте разбиение исходных данных на k-блоков со стратификацией, число блоков возьмите равным 5. Выведите индексы обучающих и тестовых частей каждого блока. Выведите для полученных блоков обучающих данных количество объектов каждого класса, сравните результаты с количеством объектов каждого класса для целевой переменной в целом.

Указание: Используйте класс [StratifiedKFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html) с параметрами n_splits = 5, shuffle=True, random_state=42. StratifiedKFold возвращает стратифицированные фолды, т.е. каждый блок содержит примерно такой же процент объектов каждого класса, что и полный набор.

In [33]:
skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)

In [34]:
for i, (train_index, test_index) in enumerate(skf.split(X, y)):
    X_train1, X_test1 = X[train_index], X[test_index]
    y_train1, y_test1 = y[train_index], y[test_index]
    print("Fold: {}, Длинна train: {}, Длинна test: {}".format(i, len(train_index), len(test_index)))
    print(f"  Train: index={train_index}")
    print(f"  Test:  index={test_index}")

Fold: 0, Длинна train: 142, Длинна test: 36
  Train: index=[  2   3   4   5   6   7   8   9  10  12  13  15  18  19  20  21  22  23
  24  26  27  28  29  30  31  32  34  35  36  37  38  39  41  43  44  45
  46  47  49  50  51  52  54  55  56  57  58  59  60  61  62  63  64  66
  67  68  69  70  71  72  73  74  75  77  78  80  81  82  84  85  87  88
  91  93  94  95  96  97  98  99 100 101 102 103 104 105 106 108 110 111
 112 113 114 115 116 117 118 122 123 124 125 126 128 129 130 131 133 134
 135 136 137 138 139 141 142 143 144 147 148 149 150 151 152 153 154 155
 157 158 160 161 162 163 164 165 167 168 169 171 173 174 175 177]
  Test:  index=[  0   1  11  14  16  17  25  33  40  42  48  53  65  76  79  83  86  89
  90  92 107 109 119 120 121 127 132 140 145 146 156 159 166 170 172 176]
Fold: 1, Длинна train: 142, Длинна test: 36
  Train: index=[  0   1   2   4   5   6   7   8  10  11  12  13  14  16  17  18  20  21
  22  23  25  26  27  29  30  31  32  33  34  35  36  37  38  39  40  

### 9. Сделайте k-блочную перекрёстную проверку (k = 5) со стратификацией модели логистической регрессии для двух метрик accuracy и F1-мера, используя библиотечную функцию cross_validate. Выведите полученные массивы со значениями метрик на обучающих и тестовых данных. Усредните их значения.

Указание: Для того, чтобы выводились массивы со значениями метрик на обучающих данных необходимо указать значения параметра return_train_score=True.

In [35]:
skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)

In [36]:
metrics = cross_validate(model, X, y, cv=skf, return_train_score=True, scoring=('accuracy', 'f1_macro') )
metrics

{'fit_time': array([0.32568908, 0.32290626, 0.37593102, 0.292171  , 0.33431411]),
 'score_time': array([0.00125289, 0.00123167, 0.00124502, 0.00135589, 0.00120282]),
 'test_accuracy': array([0.94444444, 0.97222222, 0.97222222, 0.94285714, 1.        ]),
 'train_accuracy': array([0.99295775, 1.        , 0.99295775, 1.        , 0.99300699]),
 'test_f1_macro': array([0.94582471, 0.97432099, 0.97096189, 0.9484127 , 1.        ]),
 'train_f1_macro': array([0.992657  , 1.        , 0.99351722, 1.        , 0.99354138])}

In [37]:
(metrics['test_accuracy'].mean(), metrics['test_f1_macro'].mean())

(0.9663492063492063, 0.9679040560477212)

In [38]:
(metrics['train_accuracy'].mean(), metrics['train_f1_macro'].mean())

(0.9957844971929479, 0.99594312005568)

### 10. Сделайте leave-one-out разбиение данных. Выведите индексы обучающих и тестовых частей.

Указание: Используйте  класс [LeaveOneOut](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.LeaveOneOut.html).

In [39]:
loo = LeaveOneOut()
loo.get_n_splits(X)

178

In [40]:
for i, (train_index, test_index) in enumerate(loo.split(X)):
    print(f"Fold {i}:")
    print(f"  Train: index={train_index}")
    print(f"  Test:  index={test_index}")

Fold 0:
  Train: index=[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177]
  Test:  index=[0]
Fold 1:
  Train: index=[  0   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  

### 11. Сделайте перекрёстную проверку модели логистической регрессии для метрик accuracy и F1-мера, изпользуя leave-one-out разбиение. Выведите полученные массивы со значениями метрик на тестовых данных. Усредните их значения.

In [41]:
model = LogisticRegression(max_iter=5000)
metrics = cross_validate(model, X, y, cv=loo, return_train_score=True, scoring=('accuracy', 'f1_macro') )
metrics

{'fit_time': array([0.32413125, 0.33766794, 0.39321589, 0.32211399, 0.3415401 ,
        0.32879615, 0.32516909, 0.30416203, 0.32223821, 0.2870059 ,
        0.40975213, 0.51389503, 0.34371305, 0.37158513, 0.31928229,
        0.42328286, 0.38046002, 0.32097483, 0.37810493, 0.27781606,
        0.29989982, 0.34814   , 0.30224085, 0.43022704, 0.44444919,
        0.30113888, 0.32807398, 0.28462076, 0.32100296, 0.30517411,
        0.33103108, 0.400352  , 0.3356719 , 0.38878489, 0.29200101,
        0.34043813, 0.29888415, 0.36678624, 0.36236072, 0.32249999,
        0.37993407, 0.39964962, 0.40712929, 0.38854933, 0.38585019,
        0.47266197, 0.35040402, 0.34967399, 0.37719202, 0.5022161 ,
        0.39300823, 0.37002802, 0.34444094, 0.40613508, 0.33591819,
        0.3385601 , 0.33560395, 0.32642508, 0.39168572, 0.4255321 ,
        0.37111402, 0.36485291, 0.37428093, 0.37400484, 0.34607506,
        0.31294632, 0.34896708, 0.43303394, 0.34583783, 0.35074496,
        0.28862524, 0.36100483, 0.33

In [42]:
(metrics['test_accuracy'].mean(), metrics['test_f1_macro'].mean())

(0.9550561797752809, 0.9550561797752809)

### 12. Напишите свой код, осуществляющий k-блочную перекрёстную проверку (k = 5) модели логистической регрессии для метрик accuracy и F1_мера с предварительным масштабированием данных методом стандартной нормализации. Усредните полученные результаты. 

In [43]:
scaler = StandardScaler()

In [44]:
metrics = cross_validate(model, scaler.fit_transform(X), y, cv=5, return_train_score=True, scoring=('accuracy', 'f1_macro') )
metrics

{'fit_time': array([0.00386786, 0.00414729, 0.00474691, 0.005193  , 0.00538707]),
 'score_time': array([0.00128889, 0.00271988, 0.001544  , 0.00158215, 0.0017271 ]),
 'test_accuracy': array([0.97222222, 0.97222222, 1.        , 1.        , 1.        ]),
 'train_accuracy': array([1., 1., 1., 1., 1.]),
 'test_f1_macro': array([0.97178131, 0.97178131, 1.        , 1.        , 1.        ]),
 'train_f1_macro': array([1., 1., 1., 1., 1.])}

In [45]:
(metrics['train_accuracy'].mean(), metrics['train_f1_macro'].mean())

(1.0, 1.0)

In [46]:
(metrics['test_accuracy'].mean(), metrics['test_f1_macro'].mean())

(0.9888888888888889, 0.9887125220458554)

In [47]:
cross_val_score(model, scaler.fit_transform(X), y, cv=5, scoring=('accuracy')).mean()

0.9888888888888889

In [48]:
cross_val_score(model, scaler.fit_transform(X), y, cv=5, scoring=('f1_macro')).mean()

0.9887125220458554

### 13. Сделайте k-блочную перекрёстную проверку (k = 5) модели логистической регрессии для двух метрик accuracy и F1-мера с предварительным масштабированием данных методом стандартной нормализации, используя библиотечную функцию cross_validate. Усредните значения метрик.

Указание: Используйте конвейер, передайте его в функцию cross_validate в качестве параметра model.

In [49]:
pipe = Pipeline(steps = [('scaler', StandardScaler()),
                         ('lr', LogisticRegression())
                        ])

In [50]:
metrics = cross_validate(pipe, X, y, cv=5, return_train_score=True, scoring=('accuracy', 'f1_macro') )
metrics

{'fit_time': array([0.00583196, 0.00475407, 0.00602579, 0.0049088 , 0.00450706]),
 'score_time': array([0.00133991, 0.00156808, 0.00176191, 0.00163412, 0.00148988]),
 'test_accuracy': array([0.97222222, 0.97222222, 1.        , 0.97142857, 1.        ]),
 'train_accuracy': array([1., 1., 1., 1., 1.]),
 'test_f1_macro': array([0.97178131, 0.97178131, 1.        , 0.97401299, 1.        ]),
 'train_f1_macro': array([1., 1., 1., 1., 1.])}

In [51]:
(metrics['train_accuracy'].mean(), metrics['train_f1_macro'].mean())

(1.0, 1.0)

In [52]:
(metrics['test_accuracy'].mean(), metrics['test_f1_macro'].mean())

(0.9831746031746033, 0.9835151207465049)

# Кросс-валидация в задаче регрессии

### 1. Загрузите встроенный датасет o прогрессировании диабета из библиотеки sklearn.datasets. Выведите его описание. Обозначьте за _X_ двумерный numpy-массив признаков, а за _X_frame_ датафрейм признаков с соответствующими названиями колонок. Выведите размеры и первые пять строк, полученного датафрейма. Обозначьте за _y_ одномерный numpy-массив, а за _y_series_  - серию со значениями целевой переменной. Выведите количество уникальных значений целевой переменной. Убедитесь в том, что в данных нет пропущенных значений и они все числовые.
Указание: Для загрузки датасета [diabetes](https://scikit-learn.org/1.5/modules/generated/sklearn.datasets.load_wine.html) напишите diabetes = load_diabetes(). Для определения количества объектов в каждом классе используйте метод _.value_counts()_ библиотеки _pandas_.

In [86]:
diabetes = load_diabetes()
#print(diabetes.DESCR)

In [87]:
X = diabetes.data
y = diabetes.target

In [88]:
X_frame = pd.DataFrame(X, columns=diabetes.feature_names)
X_frame.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641


In [56]:
y_series = pd.Series(y)
y_series.head()

0    151.0
1     75.0
2    141.0
3    206.0
4    135.0
dtype: float64

In [57]:
y_series.value_counts()

200.0    6
72.0     6
90.0     5
178.0    5
71.0     5
        ..
73.0     1
222.0    1
86.0     1
79.0     1
57.0     1
Name: count, Length: 214, dtype: int64

In [58]:
X_frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   age     442 non-null    float64
 1   sex     442 non-null    float64
 2   bmi     442 non-null    float64
 3   bp      442 non-null    float64
 4   s1      442 non-null    float64
 5   s2      442 non-null    float64
 6   s3      442 non-null    float64
 7   s4      442 non-null    float64
 8   s5      442 non-null    float64
 9   s6      442 non-null    float64
dtypes: float64(10)
memory usage: 34.7 KB


In [59]:
y_series.unique()

array([151.,  75., 141., 206., 135.,  97., 138.,  63., 110., 310., 101.,
        69., 179., 185., 118., 171., 166., 144., 168.,  68.,  49., 245.,
       184., 202., 137.,  85., 131., 283., 129.,  59., 341.,  87.,  65.,
       102., 265., 276., 252.,  90., 100.,  55.,  61.,  92., 259.,  53.,
       190., 142., 155., 225., 104., 182., 128.,  52.,  37., 170.,  71.,
       163., 150., 160., 178.,  48., 270., 111.,  42., 200., 113., 143.,
        51., 210., 134.,  98., 164.,  96., 162., 279.,  83., 302., 198.,
        95., 232.,  81., 246., 297., 258., 229., 275., 281., 173., 180.,
        84., 121., 161.,  99., 109., 115., 268., 274., 158., 107., 103.,
       272., 280., 336., 317., 235.,  60., 174., 126., 288.,  88., 292.,
       197., 186.,  25., 195., 217., 172., 214.,  70., 220., 152.,  47.,
        74., 295., 127., 237.,  64.,  79.,  91., 116.,  86., 122.,  72.,
        39., 196., 222., 277.,  77., 191.,  73., 263., 248., 296.,  78.,
        93., 208., 108., 154., 124.,  67., 257., 26

### 2. Сделайте hold-out (на две части) разбиение данных на обучающую и тестовую части в соотношении 4:1, зафиксировав способ перемешивания данных, указав значение параметра random_state=5. Выведите количество объектов, попавших в обучающую и тестовую части. Обучите модель линейной регрессии на обучающих данных и сделайте предсказание на тестовых данных. Вычислите значения метрик коэффициент детерминации и RMSE. На основе полученных значений сделайте предположение о качестве модели.

In [60]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.2,  random_state=5)

In [61]:
(X_train.shape, X_test.shape)

((353, 10), (89, 10))

In [62]:
model = LinearRegression()

In [63]:
model.fit(X_train, y_train)

LinearRegression()

In [64]:
y_test_pred = model.predict(X_test)

In [65]:
from sklearn import metrics

In [66]:
r2 = metrics.r2_score(y_test, y_test_pred)
r2

0.5271558947230806

In [67]:
RMSE = metrics.mean_squared_error(y_test,y_test_pred) ** 0.5
RMSE

54.603912902946895

### 3. Напишите свой код, осуществляющий k-блочную перекрёстную проверку с количеством блоков k = 10 модели линейной регрессии для метрик коэффициент детерминации и RMSE. Усредните полученные результаты. Сделайте окончательный вывод о возможности пименения модели линейной регрессии к этим данным.

In [68]:
cross_val_score(model, X, y, cv=10, scoring=('r2')).mean()

0.46196024204506025

In [69]:
cross_val_score(model, X, y, cv=10, scoring=('neg_root_mean_squared_error')).mean()

-54.40468149952541

### 4. Сделайте k-блочную перекрёстную проверку (k = 10) модели линейной регрессии для двух метрик коэффициент детерминации и ошибка RMSE, используя библиотечную функцию cross_validate. Выведите полученные массивы со значениями метрик и усредните их. Сравните полученные результаты с предыдущим пунктом.

Указание: Укажите параметр scoring = ['r2', 'neg_root_mean_squared_error']

In [70]:
metrics = cross_validate(LinearRegression(), X, y, cv=10, return_train_score=True, scoring=('r2', 'neg_root_mean_squared_error') )
metrics

{'fit_time': array([0.00280905, 0.00099206, 0.00100708, 0.0008719 , 0.00100899,
        0.00085711, 0.00107026, 0.00076699, 0.00078988, 0.00076413]),
 'score_time': array([0.00082207, 0.00047684, 0.00049305, 0.00050211, 0.00052619,
        0.00062895, 0.00046301, 0.00047898, 0.00043511, 0.00042772]),
 'test_r2': array([0.5561455 , 0.23055827, 0.35357673, 0.62190752, 0.2658727 ,
        0.61819798, 0.41815142, 0.43513747, 0.43436229, 0.68569253]),
 'train_r2': array([0.51163953, 0.52909052, 0.53129986, 0.49909207, 0.5386611 ,
        0.50115254, 0.52652869, 0.52124787, 0.52846729, 0.49939448]),
 'test_neg_root_mean_squared_error': array([-50.33726431, -53.57963777, -59.26828113, -52.5281692 ,
        -59.62964048, -53.85485494, -60.79745904, -47.77383819,
        -64.21055126, -42.06711867]),
 'train_neg_root_mean_squared_error': array([-53.89533973, -53.50836295, -52.93773115, -53.66092883,
        -52.79922705, -53.5239607 , -52.77612474, -54.12936356,
        -52.18124494, -54.636170

In [71]:
(metrics['train_r2'].mean(), metrics['train_neg_root_mean_squared_error'].mean())

(0.5186573948165049, -53.404845413377714)

In [72]:
(metrics['test_r2'].mean(), metrics['test_neg_root_mean_squared_error'].mean())

(0.46196024204506025, -54.40468149952541)

### 5. Вместо модели линейной регрессии, примените полиномиальную модель второй степени. Сделайте k-блочную перекрёстную проверку (k = 10) этой модели для метрик коэффициент детерминации и ошибки RMSE с предварительным масштабированием данных min-max методом, используя библиотечную функцию cross_validate. Усредните полученное значения метрик. Используйте конвейер make_pipeline. Построенная полиномиальная модель описывает данные лучше линейной модели или нет?

In [93]:
pipe_quick = make_pipeline(
    PolynomialFeatures(degree=2, include_bias=False), 
    MinMaxScaler(), 
    LinearRegression())

In [94]:
metrics = cross_validate(pipe_quick, X, y, cv=10, return_train_score=True, scoring=('r2', 'neg_root_mean_squared_error') )
metrics

{'fit_time': array([0.00535798, 0.00400424, 0.00412798, 0.00327396, 0.00401306,
        0.00342607, 0.00359392, 0.00294304, 0.00359607, 0.0024519 ]),
 'score_time': array([0.00133419, 0.00120091, 0.0009222 , 0.0008018 , 0.0036943 ,
        0.00107312, 0.001266  , 0.00080514, 0.0007019 , 0.00071216]),
 'test_r2': array([0.45380028, 0.1932725 , 0.25246088, 0.64137876, 0.32695037,
        0.5291241 , 0.32735178, 0.14808344, 0.36161063, 0.52995257]),
 'train_r2': array([0.59811889, 0.6051166 , 0.6062771 , 0.57650908, 0.60779015,
        0.59014287, 0.60291271, 0.60428346, 0.60827639, 0.57042636]),
 'test_neg_root_mean_squared_error': array([-55.83995137, -54.86246748, -63.73541308, -51.15772823,
        -57.09525903, -59.80795921, -65.36939104, -58.67016104,
        -68.21502537, -51.44418082]),
 'train_neg_root_mean_squared_error': array([-48.89109438, -48.99901963, -48.51914279, -49.34024253,
        -48.68293733, -48.51551032, -48.33187622, -49.21184628,
        -47.56071741, -50.611737

In [95]:
(metrics['test_r2'].mean(), metrics['test_neg_root_mean_squared_error'].mean())

(0.37639853088754677, -58.619753665706604)

In [96]:
(metrics['train_r2'].mean(), metrics['train_neg_root_mean_squared_error'].mean())

(0.5969853626988524, -48.86641246075097)